# basic version:

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = "Your-pinecone-api-key"
import streamlit as st
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

# Load your Pinecone vector store
pc = Pinecone()  # will auto-read from environment variable
index_name = "rag-index"
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embedding_model)

# Load HF LLM
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256)
llm = HuggingFacePipeline(pipeline=pipe)

# Create retriever + QA chain
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# 🖥️ Streamlit UI
st.title("📚 RAG Assistant with Pinecone + HuggingFace")

query = st.text_input("Ask me a question from your documents:")

if query:
    response = qa_chain.invoke({"query": query})
    st.markdown(f"### 🤖 Answer:\n{response['result']}")

    st.markdown("### 📂 Sources:")
    for doc in response["source_documents"]:
        st.write(f"• `{doc.metadata.get('category', 'unknown')}`")